In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection
from arcgis.features import GeoAccessor

from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd

import getpass # library to get dynamically input AGOL Password

In [ ]:
class ArcGISDataAnalyzer:
    def __init__(self, target_date):
        self.password = getpass.getpass("Please enter your ArcGIS Online password: ") #Prompt the user to enter ArcGIS Online Password # password parameter to prompt user for ArcGIS Online password
        self.gis = GIS("<replace_portal_url>", "<replace_username>", self.password)  # Initialize the GIS instance
        self.target_date = datetime.strptime(target_date, "%Y-%m-%d").date()
        self.items_before_date_all_users_df = None
        self.sorted_user_details = None
        
        self.sorted_user_details = None
        
    # Define a function to calculate credits per month based on item type and size
    def _calculate_credits_per_month(self, item_type, item_size):
        if item_type == 'Feature Service':
            credits_per_10mb = 2.4
            credits_per_month = (item_size / 10) * credits_per_10mb
        elif item_type == 'Tiled Imagery':
            credits_per_gb = 1.2
            credits_per_month = (item_size / 1024) * credits_per_gb
        elif item_type == 'Dynamic Imagery':
            num_images = 100  # Example number of images
            if num_images <= 10:
                credits_per_day = 10
            elif num_images <= 100:
                credits_per_day = 20
            elif num_images <= 1000:
                credits_per_day = 40
            elif num_images <= 10000:
                credits_per_day = 80
            elif num_images <= 100000:
                credits_per_day = 160
            else:
                credits_per_day = 320
            credits_per_month = (credits_per_day * 30)  # Assuming 30 days in a month
        else:
            credits_per_month = (item_size / 10) * 2.4  # Default value if item type is not found
        return credits_per_month

    def get_items_before_date_all_users(self):
        
        # Retrieve all users in the organization
        all_users = self.gis.users.search(query="*")  # You can modify the query as needed

        # Create a list to store item information
        all_items = []

        # Iterate through each user and retrieve their items
        for user in all_users:
            user_items = user.items()
            for item in user_items:
                # Convert timestamp to datetime
                last_viewed = item.lastViewed
                if last_viewed == -1:
                    last_viewed = datetime.utcfromtimestamp(0)  # Set to epoch time
                else:
                    last_viewed = datetime.fromtimestamp(last_viewed / 1000)  # Convert milliseconds to seconds

                # Convert datetime to ArcGIS Online format date (e.g., "yyyy-mm-ddTHH:mm:ssZ")
                last_viewed_agol_format = last_viewed.strftime("%Y-%m-%d")

                if last_viewed.date() <= self.target_date:
                    # Calculate credits per month based on item type and size
                    item_type = item.type  # Assuming the 'type' attribute is present in the item object
                    item_size = item.size / (1024 * 1024)  # Convert bytes to megabytes
                    credits_per_month = self._calculate_credits_per_month(item_type, item_size)

                    item_info = {
                        "item_name": item.title,
                        "item_id": item.itemid,
                        "item_url": item.url,
                        "last_viewed_date": last_viewed_agol_format,
                        "username": user.username,
                        "full_name": user.fullName,
                        "email": user.email,
                        "item_size_mb": item_size,
                        "item_type": item_type,
                        "estimated_credits_per_month": credits_per_month,
                        "estimated_cost_per_month": credits_per_month * 0.1
                    }
                    all_items.append(item_info)

        # Convert the list of items to a pandas DataFrame
        items_df = pd.DataFrame(all_items)

        return items_df

    def get_sorted_user_details(self):
        # Use self.gis to access the GIS instance
        gis = self.gis
        items_before_date_all_users_df = self.get_items_before_date_all_users()
        # Extract unique usernames
        unique_usernames = items_before_date_all_users_df['username'].unique()

        # Group the DataFrame by 'username' and calculate the total counts of items owned by each user
        user_item_counts = items_before_date_all_users_df.groupby('username')['item_id'].count()

        # Group the DataFrame by 'username' and sum the estimated credits per month, item size, and estimated cost per month for each user
        user_aggregated_data = items_before_date_all_users_df.groupby('username').agg({
            'estimated_credits_per_month': 'sum',
            'item_size_mb': 'sum',
            'estimated_cost_per_month': 'sum'
        })

        # Create a list to store user details including last login date, full name, email, item counts, total credit cost, total item size, and total cost per month
        user_details_list = []

        # Iterate through unique usernames and query the last login date, full name, email address, item counts, total credit cost, total item size, and total cost per month for each user
        for username in unique_usernames:
            user = gis.users.get(username)
            last_login_date_str = (datetime.fromtimestamp(user.lastLogin / 1000)).strftime("%Y-%m-%d")
            item_count = user_item_counts.get(username, 0)
            total_credits = user_aggregated_data.loc[username, 'estimated_credits_per_month']
            total_item_size_mb = user_aggregated_data.loc[username, 'item_size_mb']
            total_cost_per_month = user_aggregated_data.loc[username, 'estimated_cost_per_month']
            user_details = {
                "username": username,
                "last_login_date": last_login_date_str,
                "full_name": user.fullName,
                "email": user.email,
                "item_count": item_count,
                "total_credits": total_credits,
                "total_item_size_mb": total_item_size_mb,
                "total_cost_per_month": total_cost_per_month
            }
            user_details_list.append(user_details)

        # Sort the user details list by highest total cost per month
        sorted_user_details = sorted(user_details_list, key=lambda x: x['total_cost_per_month'], reverse=True)

        return pd.DataFrame(sorted_user_details)
    
    def get_users_before_target_date(self, items_df):
        
        # Convert the target_date to a datetime object
        target_date_dt = pd.to_datetime(self.target_date)
        
        # Calculate the adjusted target date by subtracting 6 months
        adjusted_target_date = target_date_dt - pd.DateOffset(months=6)
        
        # Get the sorted user details
        sorted_user_details = self.get_sorted_user_details()

        # Initialize a list to store usernames with dates before the adjusted_target_date
        usernames_before_date = []
        
        # Loop through the sorted_user_details
        for user_info in sorted_user_details:
            # Convert the last_login_date in the user_info to a datetime object
            last_login_date = pd.to_datetime(user_info['last_login_date'])
            
            # Check if the last_login_date is before the adjusted_target_date
            if last_login_date < adjusted_target_date:
                usernames_before_date.append(user_info)
        
        return usernames_before_date
    


In [ ]:
# Create an instance of the ArcGISDataAnalyzer class with target date "2023-01-01"
analyzer = ArcGISDataAnalyzer("2023-01-01")

# Call the get_items_before_date_all_users() method on the analyzer instance
items_df = analyzer.get_items_before_date_all_users()

items_df.head()

In [ ]:
df2 = analyzer.get_sorted_user_details()
df2

In [ ]:
df3 = analyzer.get_users_before_target_date()

In [ ]:

class CreateFeatureLayer:
    def __init__(self, analyzer: ArcGISDataAnalyzer):
        self.gis = GIS("https://esriaiddev.maps.arcgis.com/", "ralouta.aiddev", "RamiW0rk$@esri0316")  # Initialize the GIS instance
        self.analyzer = analyzer
        self.layer_name = "ArcGIS Items and Users Management"

    def create_feature_layer(self):
        # Check if feature layer with the specified name exists in ArcGIS Online
        search_results = self.gis.content.search(query=self.layer_name, item_type="Feature Layer")
        if search_results:
            # Delete the existing feature layer
            for item in search_results:
                item.delete()

        # Create a new feature layer collection
        flc = self.gis.content.create_feature_layer_collection(self.layer_name)

        # Add the three sublayers to the feature layer collection
        flc.manager.add_to_definition({
            "name": "items_before_date_all_users",
            "type": "Feature Layer",
            "fields": GeoAccessor.from_dataframe(self.analyzer.get_items_before_date_all_users()).fields,
            "source": self.analyzer.get_items_before_date_all_users().to_dict(orient="records")
        })
        flc.manager.add_to_definition({
            "name": "sorted_user_details",
            "type": "Feature Layer",
            "fields": GeoAccessor.from_dataframe(self.analyzer.get_sorted_user_details()).fields,
            "source": self.analyzer.get_sorted_user_details().to_dict(orient="records")
        })
        flc.manager.add_to_definition({
            "name": "users_before_target_date",
            "type": "Feature Layer",
            "fields": GeoAccessor.from_dataframe(self.analyzer.get_users_before_target_date()).fields,
            "source": self.analyzer.get_users_before_target_date().to_dict(orient="records")
        })

        # Publish the feature layer collection
        flc.manager.publish()

        return flc.url

In [ ]:
df3 = analyzer.get_users_before_target_date()

# Create an instance of the CreateFeatureLayer class
feature_layer_creator = CreateFeatureLayer(analyzer)

# Create the feature layer
feature_layer_url = feature_layer_creator.create_feature_layer()

# Print the URL of the feature layer
print(f"Feature layer URL: {feature_layer_url}")